In [56]:
from _Setting import StockSetting
import tushare  as ts
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col


In [ ]:
settings = StockSetting()
spark = SparkSession.builder \
    .appName("_Test") \
    .master("local[*]") \
    .config('spark.submit.pyFiles', '/work/dev/stock/_Setting.py') \
    .getOrCreate() 

In [ ]:
data_path = settings.data_all_path+"daily_data_ALL"
daily_data_ALL = spark.read.format("json").load(data_path)

In [ ]:
data_path = settings.data_all_path+"daily_basic_data_ALL"
daily_basic_data_ALL = spark.read.format("json").load(data_path)

In [ ]:
savepath = settings.data_all_path+"daily_moneyflow_data_ALL"
daily_moneyflow_data_ALL = spark.read.format("json").load(data_path)

In [5]:
savepath = settings.datasource_path+"stock_basic_main"
stock_basic_main = spark.read.format("json").load(savepath)

In [57]:
# stock_basic_main daily_data_ALL daily_basic_data_ALL daily_moneyflow_data_ALL
stocklist = stock_basic_main.where("list_date < '20200101' and ts_code like '%.SH'")
stock_pe = daily_basic_data_ALL.where("trade_date = '20220304' and pe <>'NaN'")


In [62]:
stocklist = stocklist.alias("list").join(stock_pe.alias("pe"),stocklist.ts_code == stock_pe.ts_code).select( \
    col("pe.*") \
    ,col("list.name") \
    ,col("list.area") \
    ,col("list.industry") \
    ,col("list.market"))

stocklist.show(5)

+---------------+-----+------------+------------+------+-------+-------+------+------+---------------+---------------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|        circ_mv|close| float_share|  free_share|    pb|     pe| pe_ttm|    ps|ps_ttm|       total_mv|    total_share|trade_date|  ts_code|turnover_rate|turnover_rate_f|volume_ratio|    name|area|industry|market|
+---------------+-----+------------+------------+------+-------+-------+------+------+---------------+---------------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|4.18543037181E7| 4.38|9555777.1046|1284854.3653|0.6893|16.1065| 7.6511|0.2518|0.2076|5.30291898249E7|1.21071209646E7|  20220304|600028.SH|       0.2424|         1.8025|        0.87|中国石化|北京|石油加工|  主板|
|   9991427.2087|11.26|  887338.118| 855376.0113|0.9076|13.5262|10.3597|3.8489|3.2832|   1.47102892E7|      1306420.0|  20220304|600837.SH|       0.2986|       

In [63]:
x1 = stocklist.where("circ_mv < 100000").count() #100
x2 = stocklist.where("circ_mv > 100000 and circ_mv < 500000").count()
x3 = stocklist.where("circ_mv > 500000 and circ_mv < 1000000").count()
x4 = stocklist.where("circ_mv > 1000000 ").count()
print(x1,x2,x3,x4)

1 447 287 563


In [65]:
stocklist.where("circ_mv < 100000").show()

+-------+-----+-----------+----------+-----+-------+-------+------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|circ_mv|close|float_share|free_share|   pb|     pe| pe_ttm|    ps|ps_ttm|   total_mv|total_share|trade_date|  ts_code|turnover_rate|turnover_rate_f|volume_ratio|    name|area|industry|market|
+-------+-----+-----------+----------+-----+-------+-------+------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|76832.0|17.15|     4480.0|    4480.0|3.222|25.4252|23.9014|3.3798|2.5906|687731.9065| 40100.9858|  20220304|603992.SH|       0.8895|         0.8895|        0.98|松霖科技|福建|家居用品|  主板|
+-------+-----+-----------+----------+-----+-------+-------+------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+



In [64]:
stocklist.where("ts_code = '600538.SH'").show()

+----------+-----+-----------+----------+------+--------+--------+-------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|   circ_mv|close|float_share|free_share|    pb|      pe|  pe_ttm|     ps|ps_ttm|   total_mv|total_share|trade_date|  ts_code|turnover_rate|turnover_rate_f|volume_ratio|    name|area|industry|market|
+----------+-----+-----------+----------+------+--------+--------+-------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|277850.505|  5.6| 49616.1616|37757.2703|3.2098|917.1828|180.0205|10.9657|7.5079|293551.0749| 52419.8348|  20220304|600538.SH|       2.4029|         3.1576|        1.46|国发股份|广西|医药商业|  主板|
+----------+-----+-----------+----------+------+--------+--------+-------+------+-----------+-----------+----------+---------+-------------+---------------+------------+--------+----+--------+------+

